# 格式转换： 将prod的bio数据转换为json格式的数据

In [2]:
import json
import sys
import os
import random
from seqeval.metrics.sequence_labeling import get_entities

base_dir=os.getcwd()

def write_dict_to_file(data,file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False,indent=4)
    

def read_json_file(data_dir):
    with open(data_dir,'r',encoding='utf-8') as file:
            data=json.load(file)
    return data

In [4]:
r'''
    功能：将实体识别的BIO的数据格式转换为我们需要的数据格式
    数据的格式示例：
        [
            {
                "text":,
                'metainfo':,
                'products':[,]
            },
        ]
'''
def data_transfer(input_file,out_file=None):
    pass
    data_list=[]
    
    with open(input_file, encoding="utf-8") as f:
        words, labels = [], []
        metainfo = None
        for line in f:
            line = line.rstrip()#去除右侧空白字符
            if line.startswith("#\tpassage"):
                metainfo = line
            elif line == "":
                if words:
                        data_item={}
                        text=''
                        for i,word in enumerate(words):
                             text+=word
                             if i !=len(words)-1:
                                text+=' '
                        data_item['text']=text
                        data_item['metainfo']=metainfo
                        
                        data_item['products']=products=[]
                        
                        entities = get_entities(labels)
                        #[('Prod', 9, 9), ('Reactants', 17, 17), ('Reactants', 21, 21), ('Solvent', 25, 25), ('Temperature', 27, 29)]
                        #print(entities)
                        
                        for entity in entities:
                            entity_label=entity[0]
                            
                            if entity_label=='Prod':
                                    entity_label='Product'
                            entity_beg=entity[1]
                            entity_end=entity[2]
                            entity_words=""
                            for i in range(entity_beg,entity_end+1):
                                entity_words+=words[i]
                                if i !=entity_end:
                                    entity_words+=' '
                            products.append(entity_words)
                            
                        data_list.append(data_item)
                        words, labels = [], []
                        
                       
            else:
                cols = line.split("\t")
                words.append(cols[0]) 
                labels.append(cols[1])
                
    #print(data_list)
    write_dict_to_file(data_list,out_file)
    return data_list



In [5]:
base_dir='/data/lizh/ChemRxnExtractor-main/llm_role/data/prod'

for file_item in os.listdir(base_dir):
    file_name=os.path.join(base_dir,file_item)
    if "test" in file_name:
        data_transfer(file_name,base_dir+"/test.json")
    elif "train" in file_name:
        data_transfer(file_name,base_dir+"/train.json")
    else:
        data_transfer(file_name,base_dir+'/dev.json')